                                                                                      ETL - ITEMS

- Liberias a usar

In [5]:
import pandas as pd
import numpy as np
import ast
import json
from datetime import datetime
import pyarrow as pa
import pyarrow.parquet as pq

- Leemos el archivo items

In [24]:
rows = []
with open (r'C:\Users\Administrador\Desktop\vitual\env2\Data\australian_users_items.json', encoding= 'MacRoman') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))        
df_items = pd.DataFrame(rows)

In [7]:
df_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [8]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


- Crear una función lambda para desanidar las filas y agregar las columnas originales

In [9]:
expand_items = lambda row: pd.DataFrame(row['items']).assign(user_id=row['user_id'], items_count=row['items_count'], steam_id=row['steam_id'], user_url=row['user_url'])

- Aplicamos la función lambda a la columna 'items' y concatenamos los resultados en un nuevo DataFrame


In [10]:
expanded_items_df = pd.concat(df_items.apply(expand_items, axis=1).tolist(), ignore_index=True)

- Eliminamos columnas redundantes dado que steam_id es igual a user_id

In [11]:
expanded_items_df.drop(columns=['steam_id', 'user_url'], inplace= True)

- Eliminamos la columna items_count dado que la podemos calcular y la columna item_name la tenemos en el dataframe games

In [12]:
expanded_items_df.drop(columns=['items_count', 'item_name'], inplace=True)

- Eliminamos otra columna dado que no la necesitamos para el análisis

In [13]:
expanded_items_df.drop(columns=['playtime_2weeks'], inplace=True)

In [14]:
expanded_items_df

,item_id,playtime_forever,user_id
0,10,6.0,76561197970982479
1,20,0.0,76561197970982479
2,30,7.0,76561197970982479
3,40,0.0,76561197970982479
4,50,0.0,76561197970982479
...,...,...,...
5153204,346330,0.0,76561198329548331
5153205,373330,0.0,76561198329548331
5153206,388490,3.0,76561198329548331
5153207,521570,4.0,76561198329548331


In [15]:
expanded_items_df.iloc[:2500]

,item_id,playtime_forever,user_id
0,10,6.0,76561197970982479
1,20,0.0,76561197970982479
2,30,7.0,76561197970982479
3,40,0.0,76561197970982479
4,50,0.0,76561197970982479
...,...,...,...
2495,314680,0.0,MinxIsBetterThanPotatoes
2496,366080,0.0,MinxIsBetterThanPotatoes
2497,208650,958.0,MinxIsBetterThanPotatoes
2498,209000,0.0,MinxIsBetterThanPotatoes


In [16]:
expanded_items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 3 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           object 
 1   playtime_forever  float64
 2   user_id           object 
dtypes: float64(1), object(2)
memory usage: 117.9+ MB


El archivo es demasaido pesado asi que tuve que hacerlo mas ligero. Utilizizando paquet y eliminamando registros se llego un archivo mas simple. 

- Transformamos en entero la columna playtime_forever

In [17]:
expanded_items_df['playtime_forever'] = expanded_items_df['playtime_forever'].astype(int)

In [18]:
sampled_data = expanded_items_df.sample(frac=0.1)


In [19]:
table = pa.Table.from_pandas(sampled_data)

In [20]:
pq.write_table(table, 'items1.parquet')

In [21]:
pq.write_table(table, 'items1.parquet', compression='snappy')

In [22]:
# Especifica la ruta del archivo Parquet
ruta_archivo_parquet = 'items1.parquet'

# Lee el archivo Parquet en un DataFrame de Pandas
tabla_parquet = pq.read_table(ruta_archivo_parquet)

# Convierte la tabla Parquet en un DataFrame de Pandas
dataframe_parquet1 = tabla_parquet.to_pandas()

# Ahora puedes trabajar con el DataFrame de Pandas como desees
print(dataframe_parquet1.head())


        item_id  playtime_forever            user_id
4361873  215120                 0            Andawgz
2053929  365720                 1        draoftwmate
4548169  205790                 0        stargaze77X
3725672  273110                 0  76561198073481884
131163   261640               149            shay181


In [25]:
dataframe_parquet1.info()
dataframe_parquet1.to_csv('user_items.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 515321 entries, 4361873 to 837823
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   item_id           515321 non-null  object
 1   playtime_forever  515321 non-null  int32 
 2   user_id           515321 non-null  object
dtypes: int32(1), object(2)
memory usage: 13.8+ MB
